In [8]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")
#model = SentenceTransformer("google/embeddinggemma-300m")

# The model contains:
# 1. Tokenizer - converts text to token IDs
# 2. Transformer layers - neural network weights
# 3. Pooling layer - combines token embeddings into a single vector
# 4. Normalization layer - scales the output vector  
display(model)

# closer look at model config
print("Model components:")
for i, module in enumerate(model):
    print(f"{i}. {type(module).__name__}\n")
    if 'auto_model' in dir(module):
        display(module.auto_model.config)


SentenceTransformer(
  (0): Transformer({'max_seq_length': 32768, 'do_lower_case': False, 'architecture': 'Qwen3Model'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': True, 'include_prompt': True})
  (2): Normalize()
)

Model components:
0. Transformer



Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "float32",
  "eos_token_id": 151643,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_wi

1. Pooling

2. Normalize



In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
for model_name in ["Qwen/Qwen3-Embedding-0.6B", "google/embeddinggemma-300m"]:
    model = SentenceTransformer(model_name)
    print(f"\nModel: {model_name}")
    # The model has an embedding layer with vectors for each token in vocabulary
    vocab_size = model[0].auto_model.config.vocab_size
    embedding_dim = model[0].auto_model.config.hidden_size
    max_context_size = model[0].auto_model.config.max_position_embeddings
    # check related tokenizer info: can have minimal differences for special tokens like padding, reserved tokens, etc.
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer_vocab_size = len(tokenizer)
    print(f"Embedding vocabulary size: {vocab_size:,}s; tokenizer vocab size: {len(tokenizer.get_vocab()):,}")
    print(f"Each token embedding dimension: {embedding_dim}")
    print(f"Max context size: {max_context_size} input tokens; tokenizer max length: {tokenizer.model_max_length} sequences")
    print(f"Total parameters: {model[0].auto_model.num_parameters():,} parameters")
    print(f"Estimated model size: {model[0].auto_model.num_parameters() * 4 / (1024**3):.2f} GB (float32)")
    


Model: Qwen/Qwen3-Embedding-0.6B
Embedding vocabulary size: 151,669 tokens; tokenizer vocab size: 151,669 tokens
Each token embedding dimension: 1024
Max context size: 32768 input tokens; tokenizer max length: 131072 sequences
Total parameters: 595,776,512 parameters
Estimated model size: 2.22 GB (float32)

Model: google/embeddinggemma-300m
Embedding vocabulary size: 262,144 tokens; tokenizer vocab size: 262,145 tokens
Each token embedding dimension: 768
Max context size: 2048 input tokens; tokenizer max length: 2048 sequences
Total parameters: 302,863,104 parameters
Estimated model size: 1.13 GB (float32)


In [ ]:
# analize max context size impact on embeddings
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
for model_name in ["Qwen/Qwen3-Embedding-0.6B", "google/embeddinggemma-300m"]:
    model = SentenceTransformer(model_name)
    print(f"\nModel: {model_name}")
    max_context_size = model[0].auto_model.config.max_position_embeddings

    # test with different length texts
    short_text = "Hello world"
    long_text = " ".join(["word"] * 10000) 

    # tokenize to see analize actual token count
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    short_tokens = tokenizer.encode(short_text)
    long_tokens = tokenizer.encode(long_text)    
    print(f"Short text token count: {len(short_tokens)}; decoded: {tokenizer.decode(short_tokens)}")
    print(f"Long text token count: {len(long_tokens)} (before truncation, if needed)")
    print(f"Long text will be truncated to: {max_context_size} tokens")   

    # The model will automatically truncate
    embedding = model.encode([short_text, long_text]) # one can be truncated

    print(f"\nembeddings preserve always its output dimension: {embedding.shape}")    
    for i, emb in enumerate(embedding):
        print(f"embedding {i} dimension: {emb.shape}")




Model: Qwen/Qwen3-Embedding-0.6B
Short text token count: 3; decoded: Hello world<|endoftext|>
Long text token count: 10001 (before truncation, if needed)
Long text will be truncated to: 32768 tokens

embeddings preserve always its output dimension: (2, 1024)
embedding 0 dimension: (1024,)
embedding 1 dimension: (1024,)

Model: google/embeddinggemma-300m


Token indices sequence length is longer than the specified maximum sequence length for this model (10002 > 2048). Running this sequence through the model will result in indexing errors


Short text token count: 4; decoded: <bos>Hello world<eos>
Long text token count: 10002 (before truncation, if needed)
Long text will be truncated to: 2048 tokens

embeddings preserve always its output dimension: (2, 768)
embedding 0 dimension: (768,)
embedding 1 dimension: (768,)


## handling long documents
1. Truncation (default): simple but loses information -> 1 embedding
2. Split into chunks -> n embeddings
3. Average embeddings (generally works well in search/retrieval) -> 1 embedding
4. Weighted average (head/tail chunks more important, e.g., introduction and conclusion, can be used in classification) -> 1 embedding
5. Max pooling (take maximum value across all chunks, capturing the most salient features/signals): useful when looking for specific features -> 1 embedding
6. Semantic splitting (by sentences/paragraphs) preserves context better -> n embeddings

In [ ]:
# Strategy for handling long documents
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import numpy as np

model_name = "google/embeddinggemma-300m"
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_context_size = model[0].auto_model.config.max_position_embeddings

# Example: Very long document
long_document = "This is a very long document. " * 1000

# STRATEGY 1: Simple truncation (what model.encode does by default)
# Only the first max_context_size tokens are used
embedding_truncated = model.encode(long_document)
print(f"Strategy 1 - Truncation: {embedding_truncated.shape}")
print("⚠️ Warning: Information at the end of the document is lost!\n")

# STRATEGY 2: Split into chunks and average embeddings
def chunk_text_by_tokens(text, tokenizer, max_tokens, overlap=50):
    """Split text into chunks that fit within max_tokens"""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    
    return chunks

# Split document into chunks
chunks = chunk_text_by_tokens(long_document, tokenizer, max_context_size - 10)  # -10, overlap for special tokens
print(f"Strategy 2 - Chunking: Document split into {len(chunks)} chunks")

# Embed each chunk
chunk_embeddings = model.encode(chunks)
print(f"Chunk embeddings shape: {chunk_embeddings.shape}")
print("✓ All informations are preserved\n")

# STRATEGY 3: Average embeddings
# Average all chunk embeddings to get document embedding
doc_embedding_avg = np.mean(chunk_embeddings, axis=0) # arithmetic mean: avg = sum(a) / n
print(f"Strategy 3 - Averaged document embedding: {doc_embedding_avg.shape}")
print("✓ All parts of the document are represented\n")

# STRATEGY 4: Weighted average (first/last chunks more important)
weights = np.array([2.0] + [1.0] * (len(chunks) - 2) + [2.0])  # More weight to start/end
weights = weights / weights.sum()
doc_embedding_weighted = np.average(chunk_embeddings, axis=0, weights=weights) 
print(f"Strategy 4 - Weighted average: {doc_embedding_weighted.shape}\n")
print("✓ Relevant parts of document are emphasized\n")

# STRATEGY 5: Max pooling (take maximum value across all chunks)
doc_embedding_max = np.max(chunk_embeddings, axis=0)
print(f"Strategy 5 - Max pooling: {doc_embedding_max.shape}\n")
print("✓ Captures the most salient features across chunks\n")

# STRATEGY 6: Use semantic splitting (by sentences/paragraphs)
def split_by_sentences(text, max_tokens, splitter='. '):
    """Split text by sentences, grouping to fit max_tokens"""
    sentences = text.split(splitter) # arbitrary sentence splitter
    chunks = []
    current_chunk = []
    current_length = 0
    
    for sentence in sentences:
        sentence_tokens = len(tokenizer.encode(sentence, add_special_tokens=False))
        
        if current_length + sentence_tokens > max_tokens and current_chunk:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_tokens
        else:
            current_chunk.append(sentence)
            current_length += sentence_tokens
    
    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')
    
    return chunks

semantic_chunks = split_by_sentences(long_document, max_context_size - 10)
print(f"Strategy 5 - Semantic splitting: {len(semantic_chunks)} chunks")
print("✓ Preserves sentence boundaries\n")

# Compare strategies
print("=" * 50)
print("COMPARISON")
print("=" * 50)
print(f"Original (truncated): {embedding_truncated[:5]}...")
print(f"Split chunks: {chunk_embeddings[:5]}...")
print(f"Averaged chunks:      {doc_embedding_avg[:5]}...")
print(f"Weighted average:     {doc_embedding_weighted[:5]}...")
print(f"Max pooling:          {doc_embedding_max[:5]}...")
print(f"Semantic chunks:     {model.encode(semantic_chunks)[:5]}...")


Token indices sequence length is longer than the specified maximum sequence length for this model (7001 > 2048). Running this sequence through the model will result in indexing errors


Strategy 1 - Truncation: (768,)
⚠️ Warning: Information at the end of the document is lost!

Strategy 2 - Chunking: Document split into 4 chunks
Chunk embeddings shape: (4, 768)
✓ All informations are preserved

Strategy 3 - Averaged document embedding: (768,)
✓ All parts of the document are represented

Strategy 4 - Weighted average: (768,)

✓ Relevant parts of document are emphasized

Strategy 5 - Max pooling: (768,)

✓ Captures the most salient features across chunks

Strategy 5 - Semantic splitting: 3 chunks
✓ Preserves sentence boundaries

COMPARISON
Original (truncated): [-0.0484592  -0.00667011  0.01031358 -0.00647743 -0.02938237]...
Split chunks: [[-0.04752773 -0.00685875  0.01023054 ...  0.05712191 -0.02502624
  -0.09391052]
 [-0.04752773 -0.00685875  0.01023054 ...  0.05712191 -0.02502624
  -0.09391052]
 [-0.04752773 -0.00685875  0.01023054 ...  0.05712191 -0.02502624
  -0.09391052]
 [-0.05410891  0.00279034  0.00609358 ...  0.05738835 -0.02744155
  -0.08676634]]...
Averaged 

In [51]:
# compare split on chunks (by sentences) vs averaging
from transformers import AutoTokenizer
import numpy as np

model_name = "google/embeddinggemma-300m"
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_context_size = model[0].auto_model.config.max_position_embeddings
_repeat = 200
# document with clearly different parts
long_document = (
    "Machine learning is a subset of artificial intelligence. " * _repeat + "###" +
    "Pizza is made with tomatoes, cheese, and dough. " * _repeat + "###" +
    "The solar system has eight planets orbiting the sun. " * _repeat
)

chunks = split_by_sentences(long_document, max_context_size, splitter='###')
print(f"Document split into {len(chunks)} chunks")
print(f"Chunk 1 preview: {chunks[0][:100]}...")
print(f"Chunk 2 preview: {chunks[1][:100]}...")
print(f"Chunk 3 preview: {chunks[2][:100]}...\n")

# Embed each chunk - EACH CHUNK HAS DIFFERENT CONTENT, SO DIFFERENT EMBEDDINGS!
chunk_embeddings = model.encode(chunks)
print(f"Chunk embeddings shape: {chunk_embeddings.shape}")
print(f"  - {len(chunks)} chunks")
print(f"  - Each chunk has {chunk_embeddings.shape[1]} dimensions\n")

# Let's see that chunks are DIFFERENT
print("Are all chunks identical? NO!")
print(f"Chunk 1 embedding (first 5 values): {chunk_embeddings[0][:5]}")
print(f"Chunk 2 embedding (first 5 values): {chunk_embeddings[1][:5]}")
print(f"Chunk 3 embedding (first 5 values): {chunk_embeddings[2][:5]}\n")

# Calculate similarity between chunks
from sklearn.metrics.pairwise import cosine_similarity
print(f"\nSimilarity between chunk 1 and 2: {cosine_similarity([chunk_embeddings[0]], [chunk_embeddings[1]])[0][0]:.4f}")
print(f"Similarity between chunk 2 and 3: {cosine_similarity([chunk_embeddings[1]], [chunk_embeddings[2]])[0][0]:.4f}")
print(f"Similarity between chunk 1 and 3: {cosine_similarity([chunk_embeddings[0]], [chunk_embeddings[2]])[0][0]:.4f}")
print("👉 Different content = different embeddings!\n")

# Now average to get ONE embedding for the ENTIRE document
doc_embedding_avg = np.mean(chunk_embeddings, axis=0)  # axis=0 means average across chunks
print(f"Averaged document embedding shape: {doc_embedding_avg.shape}")
print(f"  - Single embedding representing the ENTIRE document")
print(f"  - Still {doc_embedding_avg.shape[0]} dimensions\n")

print("=" * 60)
print("WHY AVERAGE?")
print("=" * 60)
print("Option 1: Keep all chunk embeddings separately")
print(f"  Result: {chunk_embeddings.shape} - multiple embeddings")
print("  Use case: When you want to search within document parts")
print("  Example: Find which paragraph answers a question\n")

print("Option 2: Average into one embedding")
print(f"  Result: {doc_embedding_avg.shape} - single embedding")
print("  Use case: Represent entire document as one vector")
print("  Example: Document similarity, clustering, classification\n")

# Demonstrate the difference in use cases
query = "What is machine learning?"
query_embedding = model.encode(query)

print("=" * 60)
print("USE CASE COMPARISON")
print("=" * 60)

# Use case 1: Search across chunks (keep them separate)
print("\n1. SEARCH WITHIN DOCUMENT (keep chunks separate):")
similarities = cosine_similarity([query_embedding], chunk_embeddings)[0]
best_chunk_idx = np.argmax(similarities)
print(f"Query: {query}")
print(f"Best matching chunk: Chunk {best_chunk_idx + 1}")
print(f"Similarity: {similarities[best_chunk_idx]:.4f}")
print(f"Content: {chunks[best_chunk_idx][:100]}...")

# Use case 2: Document-level similarity (use average)
print("\n2. DOCUMENT-LEVEL SIMILARITY (use averaged embedding):")
doc_similarity = cosine_similarity([query_embedding], [doc_embedding_avg])[0][0]
print(f"Query: {query}")
print(f"Document similarity: {doc_similarity:.4f}")
print("This represents how relevant the ENTIRE document is to the query\n")

print("=" * 60)
print("KEY INSIGHT")
print("=" * 60)
print("Averaging across chunks (axis=0) means:")
print("  For each dimension (0-767), take the average value across all chunks")
print("  Result: A single vector that captures the 'average meaning' of the document")
print("  NOT: Making all chunks identical!")


Token indices sequence length is longer than the specified maximum sequence length for this model (2201 > 2048). Running this sequence through the model will result in indexing errors


Document split into 3 chunks
Chunk 1 preview: Machine learning is a subset of artificial intelligence. Machine learning is a subset of artificial ...
Chunk 2 preview: Pizza is made with tomatoes, cheese, and dough. Pizza is made with tomatoes, cheese, and dough. Pizz...
Chunk 3 preview: The solar system has eight planets orbiting the sun. The solar system has eight planets orbiting the...

Chunk embeddings shape: (3, 768)
  - 3 chunks
  - Each chunk has 768 dimensions

Are all chunks identical? NO!
Chunk 1 embedding (first 5 values): [-0.01150803 -0.02419458 -0.00054021 -0.00617623 -0.02151605]
Chunk 2 embedding (first 5 values): [-0.02103425 -0.0354753  -0.01378786 -0.00224774 -0.01337189]
Chunk 3 embedding (first 5 values): [ 0.00678735 -0.00432749 -0.02354194  0.00788077 -0.03430591]


Similarity between chunk 1 and 2: 0.6268
Similarity between chunk 2 and 3: 0.6327
Similarity between chunk 1 and 3: 0.6129
👉 Different content = different embeddings!

Averaged document embedding shap